In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [2]:
# Read all data files of one category.
import os
data_dir = 'data/JulianTrajs/alive'
trajectories = []  # dimensions: trajectory index -> time step -> coordinate (x, y, z)
for filename in os.listdir(data_dir):
    print filename, '->',
    trajectory = np.genfromtxt(os.path.join(data_dir, filename))
    print trajectory.shape[0], 'time steps'
    trajectories.append(trajectory)
trajectories = np.array(trajectories)

cell001.dat -> 577 time steps
cell002.dat -> 577 time steps
cell003.dat -> 577 time steps
cell004.dat -> 577 time steps
cell005.dat -> 577 time steps
cell006.dat -> 577 time steps
cell007.dat -> 577 time steps
cell008.dat -> 577 time steps
cell009.dat -> 577 time steps
cell010.dat -> 577 time steps
cell011.dat -> 577 time steps
cell012.dat -> 577 time steps
cell013.dat -> 577 time steps
cell014.dat -> 577 time steps
cell015.dat -> 577 time steps
cell016.dat -> 577 time steps
cell017.dat -> 577 time steps
cell018.dat -> 577 time steps
cell019.dat -> 577 time steps
cell020.dat -> 577 time steps
cell021.dat -> 577 time steps
cell022.dat -> 577 time steps
cell023.dat -> 577 time steps
cell024.dat -> 577 time steps
cell025.dat -> 577 time steps
cell026.dat -> 577 time steps
cell027.dat -> 577 time steps
cell028.dat -> 577 time steps
cell029.dat -> 577 time steps
cell030.dat -> 577 time steps
cell031.dat -> 577 time steps
cell032.dat -> 577 time steps
cell033.dat -> 577 time steps
cell034.da

In [3]:
# Normalize trajectories to [-1, 1] for the LSTM network (it outputs values between -1 and 1 by default).
# TODO: Try to change the activation in the LSTM to do this without normalization.
min_value = np.min(trajectories)
max_value = np.max(trajectories)
normalized_trajectories = np.interp(trajectories, [min_value, max_value], [-1., 1.])
np.min(normalized_trajectories), np.max(normalized_trajectories)

(-1.0, 0.99999999999999978)

In [20]:
# TODO: Split into training/test data here.
# TODO: Possible to use validation data in this scenario?


# Split trajectories into short parts following. Save the next position for each part.
# See also https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

maxlen = 30  # Change this to see how strong the current position depends on the positions long ago. REPHRASE THIS
step = 3  # Change this to vary redundancy. 
mini_trajectories = []
next_position = []

for trajectory in normalized_trajectories:
    for i in range(0, len(trajectory) - maxlen, step):
        mini_trajectories.append(trajectory[i:i+maxlen] - trajectory[i])  # Set start of mini trajectory to origin.
        next_position.append(trajectory[i+maxlen] - trajectory[i])
        
mini_trajectories = np.array(mini_trajectories)
next_position = np.array(next_position)

print 'Created', len(mini_trajectories), 'data samples'

Created 11895 data samples


In [21]:
mini_trajectories[0], next_position[0]

(array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  4.24492619e-04,   2.56073140e-03,  -2.37195457e-03],
        [ -1.68487032e-02,   1.01107443e-02,  -2.81380391e-02],
        [ -1.67329952e-02,   1.17608261e-02,  -3.08212290e-02],
        [ -1.64276682e-02,   1.33972310e-02,  -3.33201151e-02],
        [ -1.58370445e-02,   1.53353374e-02,  -3.55339757e-02],
        [ -1.59937246e-02,   1.72160326e-02,  -3.28037968e-02],
        [ -1.63432447e-02,   1.86186856e-02,  -3.02652019e-02],
        [ -1.69490340e-02,   1.40433043e-02,  -3.36840768e-02],
        [ -1.71294628e-02,   5.40068532e-03,  -3.09719317e-02],
        [ -1.73330169e-02,   1.24444121e-02,  -3.96804076e-02],
        [ -1.75800561e-02,   8.02492167e-03,  -3.70387614e-02],
        [ -1.77364841e-02,   3.78263155e-03,  -3.43101702e-02],
        [ -1.75452827e-02,   8.71009264e-05,  -3.69304504e-02],
        [ -1.70209069e-02,   1.93610048e-03,  -3.35148921e-02],
        [ -1.19667656e-02,   8.93633482e

In [22]:
# Set up the network.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM

model = Sequential()

# TODO: Try stateful for the same trajectory
# TODO: Play around with output_dim here. 
model.add(LSTM(input_shape=(maxlen, 3), output_dim=30, return_sequences=True))  # TODO: Is it possible to use input_dim here and omit maxlen?
#model.add(Dropout(0.3))
model.add(LSTM(output_dim=3, return_sequences=False))
# TODO: Add Dropout and more LSTM layers. 

#model.add(Dropout(0.5))
#model.add(Dense(1))
#model.add(Activation('sigmoid'))

model.compile(loss='mean_squared_error', optimizer='rmsprop')

In [23]:
# Train the model.

model.fit(mini_trajectories, next_position, nb_epoch=5, verbose=1)

# TODO: See if GPU is better for RNN (according to keras docs)
# TODO: Play with batch_size
# TODO: Other implementation: learn on single trajectory, X = trajectory[:-1], y = trajectory[1:], then repeat this for the other trajectories
# Input and output dimension 3 in this case; does the network have to be stateful?
# TODO: maybe try displacement instead of absolute position

# TODO: How is the accuracy calculated here?

Epoch 1/5
11895/11895 [==============================] - 10s - loss: 0.0012    
Epoch 2/5
11895/11895 [==============================] - 10s - loss: 6.8274e-04    
Epoch 3/5
11895/11895 [==============================] - 10s - loss: 5.1614e-04    
Epoch 4/5
11895/11895 [==============================] - 10s - loss: 4.3312e-04    
Epoch 5/5
11895/11895 [==============================] - 10s - loss: 3.7538e-04    


In [24]:
# Test the model predictions on the training data. 
predicted = np.interp(model.predict(mini_trajectories), [-1., 1.], [min_value, max_value])
expected = np.interp(next_position, [-1., 1.], [min_value, max_value])

In [32]:
zip(predicted[100:120], expected[100:120])

[(array([ -0.47713725, -19.10751164, -18.02807261]),
  array([-10.096139 , -22.542305 , -25.0102487])),
 (array([ -2.73636023, -14.29024984, -26.91371666]),
  array([-18.307054 , -13.028345 , -33.2387054])),
 (array([-21.20935885, -23.8794894 , -25.49224121]),
  array([-24.246379 , -24.169965 , -36.6110255])),
 (array([-33.56979826, -28.32594549, -15.08789061]),
  array([-27.113701 , -21.297455 , -24.0159445])),
 (array([-29.4982831 , -20.36990889, -30.71483103]),
  array([-28.992832 , -16.029745 , -38.8122592])),
 (array([-27.27163613, -23.38424054, -24.43288685]),
  array([-15.703803 , -21.754745 , -30.6932721])),
 (array([-17.68826709, -26.43776746, -20.81371312]),
  array([ -8.35351  , -23.251935 , -18.1609388])),
 (array([-12.7457865 , -25.79536684, -20.97290209]),
  array([ -6.066881 , -22.020575 , -23.6133998])),
 (array([-12.02653754, -26.59642647, -19.6631983 ]),
  array([ -6.510496  , -20.746625  , -18.89604563])),
 (array([-13.95560003, -27.10990917, -16.51699824]),
  array(

In [26]:
# Rough measure for deviation. 7.715
np.sum(np.abs(predicted - expected)) / len(predicted) / 3.
# TODO: Compare with constant or random predictor

6.8543555095444537

In [27]:
np.sum(np.abs(expected)) / len(expected) / 3.

25.406808847973547

In [28]:
trajectories[0]

array([[   0.        ,    0.        ,    0.        ],
       [   0.19224073,    1.159683  ,   -1.0741913 ],
       [  -7.6303023 ,    4.5788708 ,  -12.742924  ],
       ..., 
       [ 169.71178   , -139.37348   ,   -6.4971753 ],
       [ 170.10865   , -140.88685   ,   -4.7908792 ],
       [ 172.87403   , -140.08097   ,   -5.8794939 ]])

In [29]:
# TODO: Other possible idea: Make matrix around current position with probabilities as output
# TODO: CLassification: feed in points step by step, see how classification probability changes
# TODO: Make the network predict persistence etc

In [30]:
# Constant predictor (take last position of trajectory as prediction for next position).
last_positions = mini_trajectories[:, -1]
constant_predicted = np.interp(last_positions, [-1., 1.], [min_value, max_value])
np.sum(np.abs(constant_predicted - expected)) / len(predicted) / 3.

1.8173631071746126